In [1]:
import os
import spacy
import pickle
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from scipy.optimize import linear_sum_assignment
from tqdm import tqdm
from rich import print
from itertools import product

spacy.require_gpu()
NER = spacy.load("en_core_web_trf")

## Format the DF

### Setence New Parsed

In [2]:
setence_new_parsed = pd.read_csv(os.path.join(os.getcwd(), 'data', 'processed', 'Conf_transcripts_new_parsed_processed.csv'))
setence_new_parsed.head()

/tmp/ipykernel_6795/745358426.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  setence_new_parsed = pd.read_csv(os.path.join(os.getcwd(), 'data', 'processed', 'Conf_transcripts_new_parsed_processed.csv'))


,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
0,1,"george paleologou, will kalutycz","Yes, generally speaking, a lot of the rebalanc...",sabahat khan,Thanks and good afternoon. Can you maybe talk ...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
1,2,will kalutycz,"Yes, it’s kind of a mixture, the Sandwich plan...",sabahat khan,"Alright, thanks. And then you mentioned that H...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
2,3,will kalutycz,"Yes, again, I mean we deal in all commodities ...",sabahat khan,Great. And then just if you could give us upda...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
3,5,george paleologou,Thanks George.,george doumet,"Hey, good afternoon guys and congrats on the q...",Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017
4,6,george paleologou,"Yes, actually, it was more -- it was not too m...",george doumet,Just looking at Alberta first year-over-year g...,Premium Brands Holding Corp. (OTC:PRBZF) Q1 20...,PRBZF,Q1,2017


### Simple Rule Functions

In [3]:
# 0. remove the question if it is too short (< 15 words)
def remove_too_short_question(txt, len_limit=15):
    return '' if len(txt.split()) < len_limit else txt
    
# 1. remove sentences if it is too short (< 5 words)
# def remove_too_short_first_sentence(txt, len_limit=10):
#     first_setence = txt.split('.')[0]
#     if len(first_setence.split()) < len_limit:
#         return '.'.join(txt.split('.')[1:])
#     else:
#         return txt
def remove_too_short_sentence_in_question(txt, len_limit=5):
    setences = txt.split('.')
    ret_setences = [cur_setence for cur_setence in setences if len(cur_setence.split()) >= len_limit]

    return '.'.join(ret_setences)

# 2. remove the setence if it contains thank you, good after noon, good morning, thank you very much, thanks
def remove_setence_contrains_special_words(txt, words_to_remove=['thank you', 'good afternoon', 'good morning', 'thank you very much', 'thanks', 'hey']):
    setences = txt.split('.')
    ret_setences = [cur_setence for cur_setence in setences if all(word not in cur_setence.lower() for word in words_to_remove)]

    return '.'.join(ret_setences)

# def identify_setence_contrains_special_words(txt, words_to_remove=['thank you', 'good afternoon', 'good morning', 'thank you very much', 'thanks']):
#     setences = txt.split('.')
#     for cur_setence in setences:
#         return any((word in cur_setence.lower() for word in words_to_remove))

# 3. remove the question if it is too long ( > 200 words)
def remove_too_long_question(txt, len_limit=200):
    return '' if len(txt.split()) >= len_limit else txt

# 4. replace all the name with 'Tim'
def replace_name_with(txt, replace_with_name='John'):
    parsed_txt = NER(txt)
    for word in parsed_txt.ents:
        if word.label_ == 'PERSON':
            txt = txt.replace(word.text, replace_with_name)
    return txt

### Setence New Parsed

setence_new_parsed = pd.read_csv(os.path.join(os.getcwd(), 'data', 'processed', 'Conf_transcripts_new_parsed_processed.csv'))
setence_new_parsed.head()

### AAPL(2018-Q1, 2018-Q2)

All Years

In [4]:
# years
np.sort(setence_new_parsed[setence_new_parsed['equity'] == 'AAPL'].year.unique())

array([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018], dtype=object)

Get Only One Year Data

In [5]:
# subset
filter_equity = setence_new_parsed['equity'] == 'AAPL'
aapl_raw = setence_new_parsed[filter_equity]
filter_year = aapl_raw['year'].apply(lambda x: x in ['2018', 2018])
aapl_raw = aapl_raw[filter_year].copy()
print(aapl_raw.shape)

(27, 9)

Apply Rules

In [6]:
aapl_raw['question'] = aapl_raw['question'].apply(remove_too_short_question)
aapl_raw = aapl_raw[aapl_raw['question'] != ''].copy()
aapl_raw = aapl_raw[aapl_raw['question'].apply(lambda x: x is not None)].copy()
print('Too short questions', aapl_raw.shape)
aapl_raw['question'] = aapl_raw['question'].apply(remove_too_long_question)
aapl_raw = aapl_raw[aapl_raw['question'] != ''].copy()
aapl_raw = aapl_raw[aapl_raw['question'].apply(lambda x: x is not None)].copy()
print('Too long questions', aapl_raw.shape)
aapl_raw['question'] = aapl_raw['question'].apply(remove_too_short_sentence_in_question).copy()
aapl_raw = aapl_raw[aapl_raw['question'] != ''].copy()
aapl_raw = aapl_raw[aapl_raw['question'].apply(lambda x: x is not None)].copy()
print('remove too short fist setences', aapl_raw.shape)
aapl_raw['question'] = aapl_raw['question'].apply(remove_setence_contrains_special_words).copy()
aapl_raw = aapl_raw[aapl_raw['question'] != ''].copy()
aapl_raw = aapl_raw[aapl_raw['question'].apply(lambda x: x is not None)].copy()
print('remove_setence_contrains_special_words', aapl_raw.shape)
aapl_raw['question'] = aapl_raw['question'].apply(replace_name_with).copy()
print('replace name', aapl_raw.shape)
aapl_raw['question'] = aapl_raw['question'].apply(remove_too_short_question)
aapl_raw = aapl_raw[aapl_raw['question'] != ''].copy()
aapl_raw = aapl_raw[aapl_raw['question'].apply(lambda x: x is not None)].copy()
print('Too short questions', aapl_raw.shape)
aapl_raw

Too short questions
(25, 9)

Too long questions
(23, 9)

remove too short fist setences
(23, 9)

remove_setence_contrains_special_words
(23, 9)

replace name
(23, 9)

Too short questions
(23, 9)

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
1857862,1,"timothy donald cook - apple, inc.","Sure. Shannon, it's Tim. As Luca mentioned ear...",shannon s. cross - cross research llc,I wanted to ask about your thoughts on sort o...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857863,3,"timothy donald cook - apple, inc.","Hi, Katy. It's Tim. The services grew 31%. We ...",kathryn lynn huberty - morgan stanley & co. llc,The services growth acceleration is really th...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857864,4,"timothy donald cook - apple, inc.","Yeah, I think my own view is that China and th...",kathryn lynn huberty - morgan stanley & co. llc,And it doesn't look like the threat of a trade...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857865,5,"timothy donald cook - apple, inc.","Well, Mike, it's Tim. Again, the great thing a...",michael j. olson - piper jaffray & co.,"Just following on the services question, I'd ...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857866,6,"timothy donald cook - apple, inc.",It's a good question. iPhone X was the most po...,michael j. olson - piper jaffray & co.,"And then any potential tariff issues aside, w...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857869,9,"timothy donald cook - apple, inc.","Yeah, good question. Let me start with India, ...","brian j. white - monness, crespi, hardt & co.,...","Yes, John, I think there is China numbers are ...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857870,11,"timothy donald cook - apple, inc.","We price for the value that we're delivering, ...",next we'll hear from wamsi mohan with bank of ...,"Tim, can you comment on the price elasticity ...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857871,13,"timothy donald cook - apple, inc.","Yeah, Jim, thanks for the question. In terms o...","jim suva - citigroup global markets, inc.",And I'll ask actually both my questions at th...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1858981,1,luca maestri,"Of course, we've been talking about the import...",shannon cross,"John, I wanted to talk a little bit about, mor...","Apple, Inc. (NASDAQ:AAPL) Q1 2018 Earnings Con...",AAPL,Q1,2018
1858982,2,timcook,The revenue growth from iPhone across all the ...,shannon cross,"And then, John, maybe could you talk a little ...","Apple, Inc. (NASDAQ:AAPL) Q1 2018 Earnings Con...",AAPL,Q1,2018


Process Index

In [7]:
# process index
for cur_query in tqdm(aapl_raw['time_text'].unique()):
    subset = aapl_raw[aapl_raw['time_text'] == cur_query]
    aapl_raw.loc[subset.index.tolist(), 'question_index'] = list(range(len(subset)))

print(aapl_raw['year'].unique())
print(aapl_raw['equity'].unique())
print(aapl_raw['quater'].unique())
aapl_raw

100%|██████████| 2/2 [00:00<00:00, 1007.76it/s]


[2018]

['AAPL']

['Q2' 'Q1']

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
1857862,0,"timothy donald cook - apple, inc.","Sure. Shannon, it's Tim. As Luca mentioned ear...",shannon s. cross - cross research llc,I wanted to ask about your thoughts on sort o...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857863,1,"timothy donald cook - apple, inc.","Hi, Katy. It's Tim. The services grew 31%. We ...",kathryn lynn huberty - morgan stanley & co. llc,The services growth acceleration is really th...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857864,2,"timothy donald cook - apple, inc.","Yeah, I think my own view is that China and th...",kathryn lynn huberty - morgan stanley & co. llc,And it doesn't look like the threat of a trade...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857865,3,"timothy donald cook - apple, inc.","Well, Mike, it's Tim. Again, the great thing a...",michael j. olson - piper jaffray & co.,"Just following on the services question, I'd ...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857866,4,"timothy donald cook - apple, inc.",It's a good question. iPhone X was the most po...,michael j. olson - piper jaffray & co.,"And then any potential tariff issues aside, w...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857869,5,"timothy donald cook - apple, inc.","Yeah, good question. Let me start with India, ...","brian j. white - monness, crespi, hardt & co.,...","Yes, John, I think there is China numbers are ...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857870,6,"timothy donald cook - apple, inc.","We price for the value that we're delivering, ...",next we'll hear from wamsi mohan with bank of ...,"Tim, can you comment on the price elasticity ...","Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1857871,7,"timothy donald cook - apple, inc.","Yeah, Jim, thanks for the question. In terms o...","jim suva - citigroup global markets, inc.",And I'll ask actually both my questions at th...,"Apple, Inc. (NASDAQ:AAPL) Q2 2018 Earnings Cal...",AAPL,Q2,2018
1858981,0,luca maestri,"Of course, we've been talking about the import...",shannon cross,"John, I wanted to talk a little bit about, mor...","Apple, Inc. (NASDAQ:AAPL) Q1 2018 Earnings Con...",AAPL,Q1,2018
1858982,1,timcook,The revenue growth from iPhone across all the ...,shannon cross,"And then, John, maybe could you talk a little ...","Apple, Inc. (NASDAQ:AAPL) Q1 2018 Earnings Con...",AAPL,Q1,2018


Format

In [8]:
# format
aapl = pd.DataFrame(
    {
        "fyear": aapl_raw.apply(
            lambda x: str(x["year"]) + "-" + str(x["quater"]), axis=1
        ),
        "pid": aapl_raw["question_index"],
        "text": aapl_raw["question"],
    }
)
aapl.reset_index(drop=True, inplace=True)
aapl

,fyear,pid,text
0,2018-Q2,0,I wanted to ask about your thoughts on sort o...
1,2018-Q2,1,The services growth acceleration is really th...
2,2018-Q2,2,And it doesn't look like the threat of a trade...
3,2018-Q2,3,"Just following on the services question, I'd ..."
4,2018-Q2,4,"And then any potential tariff issues aside, w..."
5,2018-Q2,5,"Yes, John, I think there is China numbers are ..."
6,2018-Q2,6,"Tim, can you comment on the price elasticity ..."
7,2018-Q2,7,And I'll ask actually both my questions at th...
8,2018-Q1,0,"John, I wanted to talk a little bit about, mor..."
9,2018-Q1,1,"And then, John, maybe could you talk a little ..."


###  SKX (2017 Q3 Q4)

All Years

In [9]:
# years
np.sort(setence_new_parsed[setence_new_parsed['equity'] == 'SKX'].year.unique())

array(['2007', '2008', '2009', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018'], dtype=object)

Get Only One Year Data

In [10]:
# subset
filter_equity = setence_new_parsed['equity'] == 'SKX'
skx_raw = setence_new_parsed[filter_equity]
filter_year = skx_raw['year'].apply(lambda x: x in ['2017', 2017])
skx_raw = skx_raw[filter_year].copy()
filter_quater = skx_raw['quater'].apply(lambda x: x in ['Q3', 'Q4'])
skx_raw = skx_raw[filter_quater].copy()
print(skx_raw.shape)
skx_raw

(111, 9)

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
2988,1,"david weinberg - skechers usa, inc.",Hi.,corinna van der ghinst - citigroup global mark...,Hi. Hi David and welcome to John.,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2989,2,"david weinberg - skechers usa, inc.","Yeah, I think it's fair to say that we think w...",corinna van der ghinst - citigroup global mark...,It looks like you guys had a very productive f...,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2990,3,"david weinberg - skechers usa, inc.","You know, that's very difficult because we're ...",corinna van der ghinst - citigroup global mark...,Where along that kind of 12% to 13% spectrum t...,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2991,4,"david weinberg - skechers usa, inc., john m. v...","So, I'll start with the repurchase. Obviously ...",corinna van der ghinst - citigroup global mark...,"Okay, that's fair. And then, I just had a quic...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2992,5,"john m. vandemore - skechers usa, inc.",We're not giving international percentages. I ...,corinna van der ghinst - citigroup global mark...,"Okay. Great. I'm sorry, I just, if I could sne...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
...,...,...,...,...,...,...,...,...,...
3407,52,david weinberg,Thanks.,tom nikic,"All right, all right, thanks, David, good luck...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3408,53,david weinberg,"No, no.",sam poser,"Two questions, real quick, the domestic backlo...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3409,54,david weinberg,"No, because I want to get down to the detail. ...",sam poser,"Well, can you give us a range?",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3410,55,david weinberg,It's doing very well. It's probably heading th...,sam poser,"Okay. Thank you. Then secondly, there are some...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017


Apply Rule

In [11]:
skx_raw['question'] = skx_raw['question'].apply(remove_too_short_question)
skx_raw = skx_raw[skx_raw['question'] != ''].copy()
skx_raw = skx_raw[skx_raw['question'].apply(lambda x: x is not None)].copy()
print('Too short questions', skx_raw.shape)
skx_raw['question'] = skx_raw['question'].apply(remove_too_long_question)
skx_raw = skx_raw[skx_raw['question'] != ''].copy()
skx_raw = skx_raw[skx_raw['question'].apply(lambda x: x is not None)].copy()
print('Too long questions', skx_raw.shape)
skx_raw['question'] = skx_raw['question'].apply(remove_too_short_sentence_in_question).copy()
skx_raw = skx_raw[skx_raw['question'] != ''].copy()
skx_raw = skx_raw[skx_raw['question'].apply(lambda x: x is not None)].copy()
print('remove too short fist setences', skx_raw.shape)
skx_raw['question'] = skx_raw['question'].apply(remove_setence_contrains_special_words).copy()
skx_raw = skx_raw[skx_raw['question'] != ''].copy()
skx_raw = skx_raw[skx_raw['question'].apply(lambda x: x is not None)].copy()
print('remove_setence_contrains_special_words', skx_raw.shape)
skx_raw['question'] = skx_raw['question'].apply(replace_name_with).copy()
print('replace name', skx_raw.shape)
skx_raw['question'] = skx_raw['question'].apply(remove_too_short_question)
skx_raw = skx_raw[skx_raw['question'] != ''].copy()
skx_raw = skx_raw[skx_raw['question'].apply(lambda x: x is not None)].copy()
print('Too short questions', skx_raw.shape)
skx_raw

Too short questions
(73, 9)

Too long questions
(73, 9)

remove too short fist setences
(73, 9)

remove_setence_contrains_special_words
(72, 9)

replace name
(72, 9)

Too short questions
(69, 9)

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
2989,2,"david weinberg - skechers usa, inc.","Yeah, I think it's fair to say that we think w...",corinna van der ghinst - citigroup global mark...,It looks like you guys had a very productive f...,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2990,3,"david weinberg - skechers usa, inc.","You know, that's very difficult because we're ...",corinna van der ghinst - citigroup global mark...,Where along that kind of 12% to 13% spectrum t...,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2991,4,"david weinberg - skechers usa, inc., john m. v...","So, I'll start with the repurchase. Obviously ...",corinna van der ghinst - citigroup global mark...,"And then, I just had a quick follow-up on you...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2992,5,"john m. vandemore - skechers usa, inc.",We're not giving international percentages. I ...,corinna van der ghinst - citigroup global mark...,"I'm sorry, I just, if I could sneak in one fo...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2995,9,"david weinberg - skechers usa, inc.",Yeah. I think the biggest piece of the portfol...,scott d. krasik - the buckingham research grou...,"Just a couple of questions, so the sales shift...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
...,...,...,...,...,...,...,...,...,...
3404,49,david weinberg,I think it's most to do with the softness in t...,tom nikic,"All right, I have a question about the U. You ...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3405,50,david weinberg,Yes.,tom nikic,"Right, right, I just meant would the growth ra...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3406,51,david weinberg,"No, I mean, that will come out in the Q. So I ...",tom nikic,"Okay, and sorry if I missed this. Did you by ...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3408,53,david weinberg,"No, no.",sam poser,"Two questions, real quick, the domestic backlo...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017


Process Index

In [12]:
# process index
for cur_query in tqdm(skx_raw['year'].unique()):
    subset1 = skx_raw[skx_raw['year'] == cur_query]
    for cur_query2 in subset1['quater'].unique():
        subset2 = subset1[subset1['quater'] == cur_query2]
        skx_raw.loc[subset2.index.tolist(), 'question_index'] = list(range(len(subset2)))

print(skx_raw['year'].unique())
print(skx_raw['equity'].unique())
print(skx_raw['quater'].unique())
print(len(skx_raw))
skx_raw.head(5)
skx_raw

100%|██████████| 1/1 [00:00<00:00, 408.88it/s]


['2017']

['SKX']

['Q4' 'Q3']

69

,question_index,answeree,answer,questioner,question,time_text,equity,quater,year
2989,0,"david weinberg - skechers usa, inc.","Yeah, I think it's fair to say that we think w...",corinna van der ghinst - citigroup global mark...,It looks like you guys had a very productive f...,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2990,1,"david weinberg - skechers usa, inc.","You know, that's very difficult because we're ...",corinna van der ghinst - citigroup global mark...,Where along that kind of 12% to 13% spectrum t...,"SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2991,2,"david weinberg - skechers usa, inc., john m. v...","So, I'll start with the repurchase. Obviously ...",corinna van der ghinst - citigroup global mark...,"And then, I just had a quick follow-up on you...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2992,3,"john m. vandemore - skechers usa, inc.",We're not giving international percentages. I ...,corinna van der ghinst - citigroup global mark...,"I'm sorry, I just, if I could sneak in one fo...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
2995,4,"david weinberg - skechers usa, inc.",Yeah. I think the biggest piece of the portfol...,scott d. krasik - the buckingham research grou...,"Just a couple of questions, so the sales shift...","SKECHERS USA, Inc. (NYSE:SKX) Q4 2017 Earnings...",SKX,Q4,2017
...,...,...,...,...,...,...,...,...,...
3404,28,david weinberg,I think it's most to do with the softness in t...,tom nikic,"All right, I have a question about the U. You ...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3405,29,david weinberg,Yes.,tom nikic,"Right, right, I just meant would the growth ra...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3406,30,david weinberg,"No, I mean, that will come out in the Q. So I ...",tom nikic,"Okay, and sorry if I missed this. Did you by ...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017
3408,31,david weinberg,"No, no.",sam poser,"Two questions, real quick, the domestic backlo...",Skechers USA (NYSE:SKX) Q3 2017 Earnings Confe...,SKX,Q3,2017


Format

In [13]:
skx = pd.DataFrame(
    {
        "fyear": skx_raw.apply(
            lambda x: str(x["year"]) + "-" + str(x["quater"]), axis=1
        ),
        "pid": skx_raw["question_index"],
        "text": skx_raw["question"],
    }
)
skx.reset_index(drop=True, inplace=True)
skx

,fyear,pid,text
0,2017-Q4,0,It looks like you guys had a very productive f...
1,2017-Q4,1,Where along that kind of 12% to 13% spectrum t...
2,2017-Q4,2,"And then, I just had a quick follow-up on you..."
3,2017-Q4,3,"I'm sorry, I just, if I could sneak in one fo..."
4,2017-Q4,4,"Just a couple of questions, so the sales shift..."
...,...,...,...
64,2017-Q3,28,"All right, I have a question about the U. You ..."
65,2017-Q3,29,"Right, right, I just meant would the growth ra..."
66,2017-Q3,30,"Okay, and sorry if I missed this. Did you by ..."
67,2017-Q3,31,"Two questions, real quick, the domestic backlo..."


## Calcualte Similarity

### Model

In [14]:
model = SentenceTransformer('all-MiniLM-L6-v2')

### Similarity Functions

In [15]:
def match_two_docs(doc1, doc2):
    # pairwise distance between sentences
    sim = np.dot(doc1, doc2.T)
    sim = (sim + 1) / 2 #! why?
    # print(sim.shape, min(sim.shape))
    # assignment is to minimize the distance of matched sentences
    # WMD may be used here (to do)
    row_ind, col_ind = linear_sum_assignment(1 - sim)
    # print(row_ind)
    # print(col_ind)
    # total similarity of matched sentences
    score = sim[row_ind, col_ind].sum()
    # print(doc_sim)
    # normalize by min. size of two docs
    score1 = score / min(sim.shape)
    # normalize by max. size of two docs
    score2 = score / max(sim.shape)
    score = [score1, score2]
    return score, row_ind, col_ind, sim

def format_index_score(index_a, index_b, score, normalized_score, sim_shape):
    normalized_by_min, normalized_by_max = normalized_score
    return [[i, j, k, normalized_by_min, normalized_by_max, sim_shape] for i, j, k in zip(index_a, index_b, score)]

def compute_sim(df, bert_model, window=1):
    # ! cik: ?
    # * window: year interval
    result = {}
    years = np.sort(df.fyear.unique())
    embeddings = bert_model.encode(
        df.text.tolist(),
        convert_to_tensor=False,
        normalize_embeddings=True,
        convert_to_numpy=True,
    )
    for i in range(len(years) - 1):
        temp_dict = {}
        for j in range(i + 1, len(years)):
            if int(years[j].split('-Q')[1]) - int(years[i].split('-Q')[1]) <= window:
                doc1_index = df[df.fyear == years[i]].index.tolist()
                doc2_index = df[df.fyear == years[j]].index.tolist()
                doc1 = embeddings[doc1_index]
                doc2 = embeddings[doc2_index]
                score, row_ind, col_ind, sim = match_two_docs(doc1, doc2)
                # get matched pair similarity
                matched_sim = sim[row_ind, col_ind]
                temp_dict[years[j]] = format_index_score(row_ind, col_ind, matched_sim, score, sim.shape)
        result[years[i]] = temp_dict
    return result

# structure example
# m = {
#     "2017-Q1": {
#         "2017-Q2": [[0, 0, 0.5], [1, 9, 0.6]],
#         "2017-Q3": [[2, 8, 0.9], [10, 11, 0.12]],
#     },
#     "2017-Q2": {
#         "2017-Q1": [[13, 14, 0.15], [16, 17, 0.18]],
#         "2017-Q3": [[19, 20, 0.21], [22, 23, 0.24]],
#     },
# }

# post process funcs
def pair_dict(listOflist):
    temp_dict = {}
    for i in listOflist:
        pair_name = f'{str(i[0])}-{str(i[1])}'
        temp_dict[pair_name] = i[2:]
    return temp_dict

def post_process_df(m):
    # add another layer
    for key in m:
        cur_dict = m.get(key)
        for key2 in cur_dict.keys():
            cur_dict[key2] = pair_dict(cur_dict.get(key2))
    # prepare multi-index
    tuple_list = []
    for key in m:
        cur_dict = m.get(key)
        for key2 in cur_dict.keys():
            tuple_list.extend((key, key2, key3) for key3 in cur_dict.get(key2).keys())
    index = pd.MultiIndex.from_tuples(tuple_list, names=["Year1", "Year2", "MaxSim-Question-Pair"])
    # get similarity score
    similarity_score = []
    normalized_by_min = []
    normalized_by_max = []
    doc1_doc2_shape = []
    for key in m:
        cur_dict = m.get(key)
        for key2 in cur_dict.keys():
            for key3 in cur_dict.get(key2).keys():
                similarity_score.append(cur_dict.get(key2).get(key3)[0])
                normalized_by_min.append(cur_dict.get(key2).get(key3)[1])
                normalized_by_max.append(cur_dict.get(key2).get(key3)[2])
                doc1_doc2_shape.append(cur_dict.get(key2).get(key3)[3])
    return pd.DataFrame({'Similarity Score': similarity_score, 'Normalized_by_min': normalized_by_min, 'Normalized_by_max': normalized_by_max, 'Doc1_Doc2_shape': doc1_doc2_shape}, index=index)

# search text
def search_text(the_df, search_df):
    search_df = search_df.copy()
    the_df = the_df.copy()
    question_from_1 = []
    question_from_2 = []
    for index, row in tqdm(the_df.iterrows()):
        # unpack
        year1, year2, question_pair = index
        year1, year1_quater = year1.split("-")
        year2, year2_quater = year2.split("-")
        question_1_index, question_2_index = question_pair.split("-")
        # locate question 1
        year_filter = search_df['year'].apply(lambda x: x in [year1, int(year1)])
        question_1 = search_df[
            year_filter
            & (search_df["quater"] == year1_quater)
            & (search_df["question_index"] == int(question_1_index))
        ]["question"]
        question_from_1.append(question_1.iloc[0])
        # locate question 2
        year_filter = search_df['year'].apply(lambda x: x in [year2, int(year2)])
        question_2 = search_df[
            year_filter
            & (search_df["quater"] == year2_quater)
            & (search_df["question_index"] == int(question_2_index))
        ]["question"]
        question_from_2.append(question_2.iloc[0])
    the_df["Question_Year1"] = question_from_1
    the_df["Question_Year2"] = question_from_2
    
    return the_df


### AAPL

In [16]:
aapl_sim = compute_sim(aapl, model)
aapl_sim = post_process_df(aapl_sim)
aapl_sim = search_text(aapl_sim, aapl_raw)

8it [00:00, 628.10it/s]


In [17]:
aapl_sim.loc['2018-Q1'][:10]

Similarity Score  Normalized_by_min  \
Year2   MaxSim-Question-Pair                                        
2018-Q2 1-6                           0.784021           0.762608   
        2-1                           0.776427           0.762608   
        3-7                           0.722822           0.762608   
        4-0                           0.790502           0.762608   
        5-3                           0.705007           0.762608   
        6-2                           0.711968           0.762608   
        13-5                          0.892222           0.762608   
        14-4                          0.717891           0.762608   

                              Normalized_by_max Doc1_Doc2_shape  \
Year2   MaxSim-Question-Pair                                      
2018-Q2 1-6                            0.406724         (15, 8)   
        2-1                            0.406724         (15, 8)   
        3-7                            0.406724         (15, 8)   
        4-0                            0.406724         (15, 8)   
        5-3                            0.406724         (15, 8)   
        6-2                            0.406724         (15, 8)   
        13-5                           0.406724         (15, 8)   
        14-4                           0.406724         (15, 8)   

                                                                 Question_Year1  \
Year2   MaxSim-Question-Pair                                                      
2018-Q2 1-6                   And then, John, maybe could you talk a little ...   
        2-1                   Growing double-digits of such a large revenue ...   
        3-7                    Just a follow-up on gross margin. Guidance fo...   
        4-0                   I know you don't talk about future products, w...   
        5-3                   And then how you're think about AR beyond iPho...   
        6-2                   I have a question and follow-up. You commented...   
        13-5                  I have two questions as well. Could you just t...   
        14-4                  John, maybe a question for you. When you talke...   

                                                                 Question_Year2  
Year2   MaxSim-Question-Pair                                                     
2018-Q2 1-6                    Tim, can you comment on the price elasticity ...  
        2-1                    The services growth acceleration is really th...  
        3-7                    And I'll ask actually both my questions at th...  
        4-0                    I wanted to ask about your thoughts on sort o...  
        5-3                    Just following on the services question, I'd ...  
        6-2                   And it doesn't look like the threat of a trade...  
        13-5                  Yes, John, I think there is China numbers are ...  
        14-4                   And then any potential tariff issues aside, w...

In [18]:
with open(os.path.join(os.getcwd(), 'data', 'result', 'linear_assignment_similarity', 'aapl_sim.pkl'), 'wb') as f:
    pickle.dump(aapl_sim, f)
aapl_sim.to_csv(os.path.join(os.getcwd(), 'data', 'result', 'linear_assignment_similarity', 'aapl_sim.csv'))

### SKX

In [19]:
skx_sim = compute_sim(skx, model)
skx_sim = post_process_df(skx_sim)
skx_sim = search_text(skx_sim, skx_raw)

33it [00:00, 786.61it/s]


In [20]:
skx_sim

Similarity Score  Normalized_by_min  \
Year1   Year2   MaxSim-Question-Pair                                        
2017-Q3 2017-Q4 0-29                          0.852109           0.756702   
                1-7                           0.822890           0.756702   
                2-33                          0.707862           0.756702   
                3-24                          0.759842           0.756702   
                4-28                          0.806235           0.756702   
                5-19                          0.814467           0.756702   
                6-31                          0.720169           0.756702   
                7-22                          0.813589           0.756702   
                8-35                          0.731016           0.756702   
                9-32                          0.719752           0.756702   
                10-21                         0.708292           0.756702   
                11-13                         0.848767           0.756702   
                12-4                          0.766692           0.756702   
                13-6                          0.752327           0.756702   
                14-17                         0.733320           0.756702   
                15-14                         0.868306           0.756702   
                16-27                         0.831579           0.756702   
                17-2                          0.710837           0.756702   
                18-1                          0.752201           0.756702   
                19-8                          0.758523           0.756702   
                20-25                         0.793800           0.756702   
                21-3                          0.759691           0.756702   
                22-34                         0.705535           0.756702   
                23-0                          0.723751           0.756702   
                24-20                         0.704698           0.756702   
                25-5                          0.725748           0.756702   
                26-10                         0.736252           0.756702   
                27-16                         0.728258           0.756702   
                28-11                         0.704015           0.756702   
                29-26                         0.732897           0.756702   
                30-30                         0.791905           0.756702   
                31-15                         0.685423           0.756702   
                32-18                         0.700424           0.756702   

                                      Normalized_by_max Doc1_Doc2_shape  \
Year1   Year2   MaxSim-Question-Pair                                      
2017-Q3 2017-Q4 0-29                           0.693644        (33, 36)   
                1-7                            0.693644        (33, 36)   
                2-33                           0.693644        (33, 36)   
                3-24                           0.693644        (33, 36)   
                4-28                           0.693644        (33, 36)   
                5-19                           0.693644        (33, 36)   
                6-31                           0.693644        (33, 36)   
                7-22                           0.693644        (33, 36)   
                8-35                           0.693644        (33, 36)   
                9-32                           0.693644        (33, 36)   
                10-21                          0.693644        (33, 36)   
                11-13                          0.693644        (33, 36)   
                12-4                           0.693644        (33, 36)   
                13-6                           0.693644        (33, 36)   
                14-17                          0.693644        (33, 36)   
                15-14                          0.693644        (33,

In [21]:
with open(os.path.join(os.getcwd(), 'data', 'result', 'linear_assignment_similarity', 'skx_sim.pkl'), 'wb') as f:
    pickle.dump(skx_sim, f)
skx_sim.to_csv(os.path.join(os.getcwd(), 'data', 'result', 'linear_assignment_similarity', 'skx_sim.csv'))